In [36]:
from lnb_hakatons import PROJECT_DIR
import pandas as pd

CLEAN_PATH = PROJECT_DIR / "data/cleaned/recenzijas_clean.csv"
data_df = pd.read_csv(CLEAN_PATH)

FILTERED_OUT_PATH = PROJECT_DIR / "data/cleaned/recenzijas_filtered_out.csv"
filtered_out_df = pd.read_csv(FILTERED_OUT_PATH).reset_index(drop=True)

DATA_DIR = PROJECT_DIR / "data/Mākslu kritika"
DATA_FILE = "cleaned-records-33-wide.csv"
original_data_df = pd.read_csv(DATA_DIR / DATA_FILE, sep=";")


/var/folders/ww/wjp3pnms1kzc2pfzj0tyl6zr0000gn/T/ipykernel_54327/2951374036.py:5: DtypeWarning: Columns (33,38,39,41,42,55,56,65) have mixed types. Specify dtype option on import or set low_memory=False.
  data_df = pd.read_csv(CLEAN_PATH)
/var/folders/ww/wjp3pnms1kzc2pfzj0tyl6zr0000gn/T/ipykernel_54327/2951374036.py:8: DtypeWarning: Columns (15,16,17,21,27,32,36,40,43,46,57,58,76,77,78,79,80,81) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_out_df = pd.read_csv(FILTERED_OUT_PATH).reset_index(drop=True)
/var/folders/ww/wjp3pnms1kzc2pfzj0tyl6zr0000gn/T/ipykernel_54327/2951374036.py:12: DtypeWarning: Columns (6,17,44,46,47,48,60,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  original_data_df = pd.read_csv(DATA_DIR / DATA_FILE, sep=";")


In [2]:
recenziju_tipu_skaits = (
    data_df
    .groupby("recenzijas_tips")
    .agg(skaits=("recenzijas_tips", "count"))
    .reset_index()
)
recenziju_tipu_skaits

,recenzijas_tips,skaits
0,Apskati un recenzijas,327
1,Baleta recenzijas,62
2,Dejas recenzijas,42
3,Izstāžu recenzijas,582
4,Kinofilmu recenzijas,2414
5,Literatūras recenzijas,2908
6,Mūzikas recenzijas,900
7,Operas recenzijas,392
8,Televīzijas raidījumu recenzijas,49
9,Teātra recenzijas,3067


In [262]:
tipi = [
    "Teātra recenzijas",
    "Literatūras recenzijas",
    "Kinofilmu recenzijas",
    "Mūzikas recenzijas",
    "Izstāžu recenzijas",
    "Operas recenzijas",
]
time_series = []
for tips in tipi:
    time_series.append(
        data_df
        .query("recenzijas_tips == @tips")
        .groupby("GADS (008)")
        .agg(
            skaits=("recenzijas_tips", "count"),
        )
        .reset_index()
        .rename(columns={"GADS (008)": "gads"})
        .assign(recenzijas_tips=tips)
)

time_series = pd.concat(time_series, ignore_index=True)

In [ ]:
time_series

,gads,skaits,recenzijas_tips
0,2015,384,Teātra recenzijas
1,2016,348,Teātra recenzijas
2,2017,357,Teātra recenzijas
3,2018,333,Teātra recenzijas
4,2019,325,Teātra recenzijas
...,...,...,...
62,2021,24,Operas recenzijas
63,2022,21,Operas recenzijas
64,2023,22,Operas recenzijas
65,2024,21,Operas recenzijas


In [291]:
import altair as alt

# Bar chart showing review type counts
bar_chart = alt.Chart(recenziju_tipu_skaits).mark_bar().add_selection(
    alt.selection_interval()
).encode(
    y=alt.Y('recenzijas_tips:N', 
            sort='-x'),  # Sort by count descending
    x=alt.X('skaits:Q'),
    # color=alt.Color('recenzijas_tips:N', 
                    # legend=None),
    tooltip=['recenzijas_tips:N', 'skaits:Q']
).properties(
    width=600,
    height=400,
).interactive()

bar_chart


/var/folders/ww/wjp3pnms1kzc2pfzj0tyl6zr0000gn/T/ipykernel_60866/3213979254.py:4: AltairDeprecationWarning: 
Deprecated since `altair=5.0.0`. Use add_params instead.
  bar_chart = alt.Chart(recenziju_tipu_skaits).mark_bar().add_selection(


alt.Chart(...)

In [290]:
# Line chart showing time series trends for each review type
line_chart = alt.Chart(time_series.query("gads >= 2015")).mark_line(
    strokeWidth=3,
    point=True
).encode(
    x=alt.X(
        'gads:O', 
        # title='gads',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y('skaits:Q', 
            scale=alt.Scale(zero=False)),
    color=alt.Color('recenzijas_tips:N'),
    # strokeDash=alt.StrokeDash('recenzijas_tips:N'),
    tooltip=['gads:O', 'skaits:Q', 'recenzijas_tips:N']
).properties(
    width=700,
    height=500,
).interactive()

line_chart


alt.Chart(...)

In [ ]:
# Combined visualization: Bar chart and line chart side by side
combined_chart = alt.hconcat(
    bar_chart,
    line_chart,
    spacing=50
).resolve_scale(
    color='independent'
).properties(
    title='Review Analysis: Distribution and Trends'
)

combined_chart


In [ ]:
# Alternative: Stacked area chart for better trend comparison
area_chart = alt.Chart(time_series).mark_area(
    opacity=0.7,
    stroke='white',
    strokeWidth=1
).encode(
    x=alt.X('gads:O', 
            title='Year',
            scale=alt.Scale(zero=False)),
    y=alt.Y('skaits:Q', 
            title='Number of Reviews',
            stack='normalize'),  # Normalize to show proportions
    color=alt.Color('recenzijas_tips:N', 
                    title='Review Type',
                    scale=alt.Scale(scheme='category20')),
    tooltip=['gads:O', 'skaits:Q', 'recenzijas_tips:N']
).properties(
    width=700,
    height=400,
    title='Proportional Review Trends Over Time'
).interactive()

area_chart


# Check filtered out data

In [37]:
filtered_out_df["PRIEKŠMETS - ŽANRS (655)_a"].isna().sum()

np.int64(38417)

In [38]:
filtered_out_df["PRIEKŠMETS - ŽANRS (655)_a"].value_counts().head(20)

PRIEKŠMETS - ŽANRS (655)_a
Intervijas.                           4292
Izstādes.                             2045
Personālizstādes.                      887
Zinātniskie raksti.                    614
Kongresi, konferences utt.             252
Biogrāfijas.                            87
Latviešu esejas.                        83
Latviešu dokumentālā publicistika.      74
Grāmatu apskati.                        68
Teātra recenzijas.                      60
Dokumentālā publicistika.               43
Krājumi.                                33
Nekrologi.                              25
Latviešu dzeja.                         23
Kinofilmu recenzijas.                   10
Latviešu ceļojumu apraksti.              8
Latviešu atmiņas.                        7
Latviešu stāsti.                         6
Izstāžu recenzijas.                      6
Latviešu romāni.                         5
Name: count, dtype: int64

In [39]:
filtered_out_df["PRIEKŠMETS - TEMATS (650)_a"].value_counts().head(20)

PRIEKŠMETS - TEMATS (650)_a
Koncerti (pasākums)              3031
Teātra iestudējumi               2607
Mūzikas festivāli                1844
Fotogrāfija                      1045
Gleznotājas                       811
Mākslinieki                       786
Gleznotāji                        765
Amatierteātris                    752
Aktieri                           731
Televīzijas seriāli               660
Dziedātājas                       631
Fotogrāfi                         629
Deju kolektīvi                    625
Kinofestivāli                     625
Mūzikas grupas                    565
Latvijas dokumentālās filmas.     557
Mākslas izstādes                  554
Aktrises                          552
Mākslinieces                      525
Spēlfilmas                        499
Name: count, dtype: int64

In [48]:
# Option 1: Set pandas display options to show all content
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Show full content of each column
pd.set_option('display.width', None)  # Don't wrap output
pd.set_option('display.max_rows', None)  # Show all rows (if needed)

# Now try displaying the row again
filtered_out_df.iloc[47165]


AUTORS (100)_4                                                                                                                                                                  aut
AUTORS (100)_a                                                                                                                                                        Jance, Diāna,
AUTORS (100)_c                                                                                                                                                                  NaN
AUTORS (100)_d                                                                                                                                                                1970-
PAPILDRAKSTS (700)_4                                                                                                                                                            aqt
PAPILDRAKSTS (700)_a                                                                                

In [46]:
pd.set_option('display.max_colwidth', None)
cols = [
    "AUTORS (100)_4",
    "RAKSTA NOSAUKUMS (245)_a", 
    "RAKSTA NOSAUKUMS (245)_b",
    "PRIEKŠMETS - ŽANRS (655)_a",
    "PRIEKŠMETS - ŽANRS (655)_x",
    "PRIEKŠMETS - TEMATS (650)_a",
    "filter_reason",
]
filtered_out_df[filtered_out_df["PRIEKŠMETS - TEMATS (650)_a"] == "Teātra iestudējumi"][cols]

,AUTORS (100)_4,RAKSTA NOSAUKUMS (245)_a,RAKSTA NOSAUKUMS (245)_b,PRIEKŠMETS - ŽANRS (655)_a,PRIEKŠMETS - ŽANRS (655)_x,PRIEKŠMETS - TEMATS (650)_a,filter_reason
46,re'v,Bezceļa stopētāji :,"[krievu dramaturga un scenārista Aleksandra Molčanova luga ""Pusslepkava"" (rež. Regnārs Vaivars) Dailes teātrī] /",Teātra recenzijas.,NaN,Teātra iestudējumi,Author type not 'aut' or 'rev'
761,NaN,Шанс на счастье :,"[о спектакле ""Она в отсутствии любви и смерти"" (режиссеры Андрей Гаркави, Татьяна Черковская) по одноим. пьесе рос. писателя Эдварда Радзинского в постановке арт-платформы ""Stadia"" в Риж. Малом театре] /",Teātra recenzijas.,NaN,Teātra iestudējumi,Author type not 'aut' or 'rev'
1165,NaN,Turpinājums izrādei :,[aktieru izteikumi par kritiķiem un kritiku] /,NaN,NaN,Teātra iestudējumi,Author type not 'aut' or 'rev'
1591,ive,Reiz bija profesionāls teātris Jelgavā :,[par Jelgavas profesionālajam teātrim velt. izstādi Ādolfa Alunāna Memoriālajā muzejā : saruna ar muzeja vadītāju] /,NaN,NaN,Teātra iestudējumi,Author type not 'aut' or 'rev'
2480,ive,"""Opera ir karaļu māksla"" :","[saruna ar Lietuvas Nacionālā operas un baleta teātra ģenerāldirektoru, operdziedātāju, komp., muzikālā teātra prod. : sakarā ar Latvijas Nacionālās operas un baleta un Lietuvas Nacionālā operas un baleta teātra apmaiņas viesizrādēm Rīgā un Viļņā] /",Intervijas.,NaN,Teātra iestudējumi,Author type not 'aut' or 'rev'
...,...,...,...,...,...,...,...
47163,aut,Blaumanis visu ir paredzējis :,[par jaunās teātra sezonas jauniestudējumiem Latvijas teātros] /,NaN,NaN,Teātra iestudējumi,"Not a review, book review, or history/criticism"
47164,aut,Draugi! Būs slikti! :,[par jaunās teātra sezonas jauniestudējumiem Latvijas teātros] /,NaN,NaN,Teātra iestudējumi,"Not a review, book review, or history/criticism"
47165,aut,"Straumēnu DNS ""Billītēs"" :","[par Cēsu Mazo teātri, izrādi ""Bezgalīgais ritums"" (režisore Liena Šmukste) pēc rakstnieka Edvarta Virzas poēmas ""Straumēni"" motīviem] /",NaN,NaN,Teātra iestudējumi,"Not a review, book review, or history/criticism"
47167,aut,No lībiešiem līdz kvadroberiem :,[par nevalstisko teātru gaidāmo sezonu] /,NaN,NaN,Teātra iestudējumi,"Not a review, book review, or history/criticism"


In [47]:
filtered_out_df.iloc[47165]

AUTORS (100)_4                                                                   aut
AUTORS (100)_a                                                         Jance, Diāna,
AUTORS (100)_c                                                                   NaN
AUTORS (100)_d                                                                 1970-
PAPILDRAKSTS (700)_4                                                             aqt
                                                          ...                       
ĢEOGRĀFISKAIS NOSAUKUMS - 2 (651)     $$aSalgales pagasts (Jelgavas novads, Latvija)
ĢEOGRĀFISKAIS NOSAUKUMS - 3 (651)                                                NaN
ĢEOGRĀFISKAIS NOSAUKUMS - 4 (651)                                                NaN
ĢEOGRĀFISKAIS NOSAUKUMS - 5 (651)                                                NaN
filter_reason                        Not a review, book review, or history/criticism
Name: 47165, Length: 89, dtype: object

In [14]:
[col for col in original_data_df.columns if "PRIEKŠMETS - ŽANRS (655)" in col]

['PRIEKŠMETS - ŽANRS (655)']

In [19]:
original_data_df["PRIEKŠMETS - ŽANRS (655)"].isna().sum()

np.int64(38417)